<a href="https://colab.research.google.com/github/Tech-crafted/Build-RNN/blob/main/RNN_with_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import random
with open("/content/data.py", "r") as file:
    data = file.read()
print(data)

train_data = {
  'good': True,
  'bad': False,
  'happy': True,
  'sad': False,
  'not good': False,
  'not bad': True,
  'not happy': False,
  'not sad': True,
  'very good': True,
  'very bad': False,
  'very happy': True,
  'very sad': False,
  'i am happy': True,
  'this is good': True,
  'i am bad': False,
  'this is bad': False,
  'i am sad': False,
  'this is sad': False,
  'i am not happy': False,
  'this is not good': False,
  'i am not bad': True,
  'this is not sad': True,
  'i am very happy': True,
  'this is very good': True,
  'i am very bad': False,
  'this is very sad': False,
  'this is very happy': True,
  'i am good not bad': True,
  'this is good not bad': True,
  'i am bad not good': False,
  'i am good and happy': True,
  'this is not good and not happy': False,
  'i am not at all good': False,
  'i am not at all bad': True,
  'i am not at all happy': False,
  'this is not at all sad': True,
  'this is not at all happy': False,
  'i am good right now': True,
  'i 

In [6]:
from data import train_data, test_data
vocab = list(set([w for text in train_data.keys() for w in text.split(' ')]))
vocab_size = len(vocab)
print('%d unique words found' % vocab_size)

18 unique words found


In [7]:
word_to_idx = { w: i for i, w in enumerate(vocab) }
idx_to_word = { i: w for i, w in enumerate(vocab) }
# print(word_to_idx['good'])
# print(idx_to_word[0])

In [8]:
def createInputs(text):
    inputs = []
    for w in text.split(' '):
        if w in word_to_idx:  # check first
            v = np.zeros((vocab_size, 1))
            v[word_to_idx[w]] = 1
            inputs.append(v)
    return inputs

In [9]:
import numpy as np

class RNN:
    def __init__(self, input_size, output_size, hidden_size=64):
        self.Whh = np.random.randn(hidden_size, hidden_size) / 1000
        self.Wxh = np.random.randn(hidden_size, input_size) / 1000
        self.Why = np.random.randn(output_size, hidden_size) / 1000
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    def forward(self, inputs):
        h = np.zeros((self.Whh.shape[0], 1))
        self.last_inputs = inputs
        self.last_hs = { 0: h }
        for i, x in enumerate(inputs):
            h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
            self.last_hs[i + 1] = h
        y = self.Why @ h + self.by
        return y, h
    def backprop(self,d_y,learn_rate=2e-2):
        n = len(self.last_inputs)
        d_Why = d_y @ self.last_hs[n].T
        d_by = d_y
        d_Whh = np.zeros(self.Whh.shape)
        d_Wxh = np.zeros(self.Wxh.shape)
        d_bh = np.zeros(self.bh.shape)
        d_h = self.Why.T @ d_y
        for t in reversed(range(n)):
          temp = ((1-self.last_hs[t+1]**2)*d_h)
          d_bh += temp
          d_Whh += temp @ self.last_hs[t].T
          d_Wxh += temp @ self.last_inputs[t].T
          d_h = self.Whh @ temp
        for d in [d_Wxh, d_Whh, d_Why, d_bh, d_by]:
          np.clip(d, -1, 1, out=d)
        self.Whh -= learn_rate * d_Whh
        self.Wxh -= learn_rate * d_Wxh
        self.Why -= learn_rate * d_Why
        self.bh -= learn_rate * d_bh
        self.by -= learn_rate * d_by

In [10]:
def softmax(xs):
  return np.exp(xs) / sum(np.exp(xs))

rnn = RNN(vocab_size, 2)
inputs = createInputs('happy')
out, h = rnn.forward(inputs)
probs = softmax(out)
print(probs)

[[0.50000316]
 [0.49999684]]


In [11]:
def processData(data, backprop=True):
  items = list(data.items())
  random.shuffle(items)
  loss = 0
  num_correct = 0
  for x, y in items:
    inputs = createInputs(x)
    target = int(y)
    out, _ = rnn.forward(inputs)
    probs = softmax(out)
    loss -= np.log(probs[target])
    num_correct += int(np.argmax(probs) == target)
    if backprop:
      d_L_d_y = probs
      d_L_d_y[target] -= 1
      rnn.backprop(d_L_d_y)
  return loss / len(data), num_correct / len(data)

In [12]:
# Training loop
for epoch in range(1000):
  train_loss, train_acc = processData(train_data)

  if epoch % 100 == 99:
    print('--- Epoch %d' % (epoch + 1))
    print('Train:\tLoss %.3f | Accuracy: %.3f' % (train_loss, train_acc))
    test_loss, test_acc = processData(test_data, backprop=False)
    print('Test:\tLoss %.3f | Accuracy: %.3f' % (test_loss, test_acc))

--- Epoch 100
Train:	Loss 0.689 | Accuracy: 0.552
Test:	Loss 0.697 | Accuracy: 0.500


/tmp/ipython-input-12-3022058651.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Train:\tLoss %.3f | Accuracy: %.3f' % (train_loss, train_acc))
/tmp/ipython-input-12-3022058651.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Test:\tLoss %.3f | Accuracy: %.3f' % (test_loss, test_acc))


--- Epoch 200
Train:	Loss 0.672 | Accuracy: 0.638
Test:	Loss 0.718 | Accuracy: 0.450
--- Epoch 300
Train:	Loss 0.561 | Accuracy: 0.707
Test:	Loss 0.639 | Accuracy: 0.700
--- Epoch 400
Train:	Loss 0.471 | Accuracy: 0.776
Test:	Loss 0.682 | Accuracy: 0.650
--- Epoch 500
Train:	Loss 0.226 | Accuracy: 0.879
Test:	Loss 0.380 | Accuracy: 0.800
--- Epoch 600
Train:	Loss 0.061 | Accuracy: 1.000
Test:	Loss 0.127 | Accuracy: 0.950
--- Epoch 700
Train:	Loss 0.004 | Accuracy: 1.000
Test:	Loss 0.012 | Accuracy: 1.000
--- Epoch 800
Train:	Loss 0.002 | Accuracy: 1.000
Test:	Loss 0.005 | Accuracy: 1.000
--- Epoch 900
Train:	Loss 0.001 | Accuracy: 1.000
Test:	Loss 0.003 | Accuracy: 1.000
--- Epoch 1000
Train:	Loss 0.001 | Accuracy: 1.000
Test:	Loss 0.002 | Accuracy: 1.000


In [13]:
def predict(text):
    inputs = createInputs(text)
    out, _ = rnn.forward(inputs)
    probs = softmax(out)

    if np.argmax(probs) == 1:
        print("True")
    else:
        print("False")

In [14]:
predict('I am very sad')

False
